# 数据加载，存储与文件格式

## 读写文本格式的数据

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [3]:
!cat ch06/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
df = pd.read_csv('ch06/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
pd.read_table('ch06/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
help(pd.read_)

Help on function read_table in module pandas.io.parsers:

read_table(filepath_or_buffer, sep='\t', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, iterator=False, chunksize=None, compression='infer', thousands=None, decimal=b'.', lineterminator=None, quotechar='"', quoting=0, escapechar=None, comment=None, encoding=None, dialect=None, tupleize_cols=False, error_bad_lines=True, warn_bad_lines=True, skipfooter=0, skip_footer=0, doublequote=True, delim_whitespace=False, as_recarray=False, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, memory_map=False, float_precisi

In [6]:
!cat ch06/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [7]:
pd.read_csv('ch06/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('ch06/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ch06/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat ch06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv('ch06/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
list(open('ch06/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('ch06/ex3.txt',sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [14]:
!cat ch06/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [15]:
pd.read_csv('ch06/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [16]:
!cat ch06/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [17]:
result = pd.read_csv('ch06/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [7]:
result = pd.read_csv('ch06/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [21]:
pd.read_csv('ch06/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

In [22]:
result = pd.read_csv('ch06/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [23]:
pd.read_csv('ch06/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [24]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)
chunker

In [25]:
chunker = pd.read_csv('ch06/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [26]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 将数据写出到文本格式

In [9]:
data = pd.read_csv('ch06/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [10]:
data.to_csv('ch06/out.csv')

In [29]:
!cat ch06/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [30]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [31]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [32]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [33]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [34]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('ch06/tseries.csv')
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 手工处理分隔符格式

In [35]:
!cat ch06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [36]:
import csv
f = open('ch06/ex7.csv')

reader = csv.reader(f)

In [37]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [38]:
with open('ch06/ex7.csv') as f:
    lines = list(csv.reader(f))

In [39]:
header, values = lines[0], lines[1:]

In [40]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [41]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [42]:
#reader = csv.reader(f, dialect=my_dialect)
#reader = csv.reader(f, delimiter='|')

In [43]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## JSON数据

In [13]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [14]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [15]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [47]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


## XML和HTML：Web信息收集

In [48]:
#conda install lxml pip install beautifulsoup4 html5lib

In [49]:
from lxml.html import parse 
from urllib.request import urlopen #与书上不同，采用python3的写法
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()

In [50]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x1055e0458>,
 <Element a at 0x1055e04a8>,
 <Element a at 0x1055e04f8>,
 <Element a at 0x1055e0548>,
 <Element a at 0x1055e0598>]

In [51]:
lnk = links[28]
lnk

<Element a at 0x1055e0868>

In [52]:
lnk.get('href')

'/quote/AAPL/options?strike=false&straddle=false'

In [53]:
lnk.text_content()

'35.00'

In [54]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['/',
 '/watchlists',
 '/portfolios',
 '/screener',
 '/calendar',
 '/industries',
 '/personal-finance',
 '/tech',
 '/topic/yahoo-finance-podcast',
 'https://www.yahoo.com/author/andy-serwer']

In [55]:
tables = doc.findall('.//table') 
tables


[<Element table at 0x1029a1cc8>, <Element table at 0x1055e6e08>]

In [56]:
#代码与书上有差异
calls = tables[0] 
puts = tables[1]
rows = calls.findall('.//tr')
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]
_unpack(rows[0], kind='th')

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [57]:
_unpack(rows[1], kind='td')

['AAPL180921C00002500',
 '2018-09-14 1:13PM EDT',
 '2.50',
 '220.65',
 '220.80',
 '221.55',
 '-1.95',
 '-0.88%',
 '2',
 '124',
 '1,831.25%']

In [58]:
from pandas.io.parsers import TextParser
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

call_data = parse_options_data(calls)

put_data = parse_options_data(puts)

call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL180921C00002500,2018-09-14 1:13PM EDT,2.5,220.65,220.80,221.55,-1.95,-0.88%,2,124,"1,831.25%"
1,AAPL180921C00005000,2018-09-13 10:28AM EDT,5.0,220.10,218.30,219.00,0.00,-,59,59,"1,420.31%"
2,AAPL180921C00007500,2018-09-14 11:46PM EDT,7.5,217.60,215.85,216.55,0.00,-,93,93,"1,290.63%"
3,AAPL180921C00010000,2018-09-14 11:45PM EDT,10.0,215.10,213.35,214.05,0.00,-,57,57,"1,167.19%"
4,AAPL180921C00012500,2018-09-13 10:22AM EDT,12.5,212.15,210.85,211.55,0.00,-,12,22,"1,073.44%"
5,AAPL180921C00015000,2018-09-13 10:22AM EDT,15.0,209.65,208.35,209.05,0.00,-,7,57,"1,000.00%"
6,AAPL180921C00030000,2018-09-14 11:46PM EDT,30.0,194.90,193.35,194.05,0.00,-,29,29,732.81%
7,AAPL180921C00035000,2018-09-14 11:46PM EDT,35.0,190.25,188.35,189.05,0.00,-,50,50,675.78%
8,AAPL180921C00040000,2018-09-14 11:46PM EDT,40.0,185.25,183.35,184.05,0.00,-,20,20,627.34%
9,AAPL180921C00045000,2018-09-14 11:46PM EDT,45.0,180.25,178.35,179.05,0.00,-,50,50,584.38%


### 利用lxml.objectify解析XML

In [20]:
from lxml import objectify
path = 'student.xml'
parsed = objectify.parse(open(path)) 
root = parsed.getroot()
root

<Element 学生们 at 0x105f82048>

In [28]:
data = []
for student in root.getchildren(): 
    print(student['姓名'].pyval)

张三
李四


In [17]:
from lxml import objectify
path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path)) 
root = parsed.getroot()
root

<Element PERFORMANCE at 0x105f5dc08>

In [60]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']
for elt in root.INDICATOR: 
    el_data = {}
    for child in elt.getchildren(): 
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [61]:
perf = pd.DataFrame(data)
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [62]:
from io import StringIO #与书上不同，支持python3
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [63]:
root

<Element a at 0x107490288>

In [64]:
root.get('href')

'http://www.google.com'

In [65]:
root.text

'Google'

## 二进制数据格式

In [66]:
frame = pd.read_csv('ch06/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [67]:
#frame.save('ch06/frame_pickle') 这个会报错
frame.to_pickle('ch06/frame_pickle')

In [68]:
#pd.load('ch06/frame_pickle') 这个会报错
pd.read_pickle('ch06/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## 使用HDF5格式

In [76]:
#若无法执行需要安装tables库
#执行pip install tables
store = pd.HDFStore('mydata.h5','w') #以写入模式打开文件
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [77]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [78]:
store.close() #操作完成后请务必关闭

## 读取Microsoft Excel文件

In [79]:
#若无法执行，安装xlrd
#执行pip3 install xlrd
xls_file = pd.ExcelFile('ex1.xlsx') 
#作者的数据集换名字了

In [80]:
table = xls_file.parse('Sheet1')

# 使用HTML和Web API

In [81]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [82]:
import json
data = resp.json()
data[0]['title']
#书上的代码没有对应数据，这是作者给的代码

'TST/CLN: Fixturize frame/test_analytics'

In [84]:
data = json.loads(resp.text)
data

[{'assignee': None,
  'assignees': [],
  'author_association': 'CONTRIBUTOR',
  'body': '- [x] 1 step closer towards #22471\r\n- [x] tests added / passed\r\n- [x] passes `git diff upstream/master -u -- "*.py" | flake8 --diff`\r\n\r\nThis module is **much** harder to fixturize than e.g. #22236 or #22730, mainly due to the class methods `_check_stat_op` and `_check_bool_op`, which, despite having an argument for the frame they\'re testing, are *also* testing on other quasi-fixtures of `TestData`. Since I can\'t import directly from `frame/conftest` without getting `RemovedInPytest4Warnings`, I made theses fixtures explicit arguments of the respective methods.\r\n\r\nFurthermore, I extracted two fixtures that basically correspond to those methods being called without a `frame` argument, and added them to `conftest`.\r\n\r\nThe larger question is how to avoid all these redundant calls being made (e.g. in `test_max`), and how `_check_stat_op` / `_check_stat_op` should be properly split up i

In [86]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,22733,TST/CLN: Fixturize frame/test_analytics,[],open
1,22732,read_html substitutions,[],open
2,22731,DOC: Fix Series nsmallest and nlargest docstri...,[],open
3,22730,TST/CLN: Fixturize frame tests,[],open
4,22729,"Groupby Get Groups fails with ""OverflowError: ...",[],open
5,22728,CLN/DOC: Refactor timeseries.rst intro and ove...,[],open
6,22727,Feature request: add an optional values parame...,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open
7,22726,CLN: io/formats/html.py: refactor indentation,"[{'id': 57395487, 'node_id': 'MDU6TGFiZWw1NzM5...",open
8,22725,CLN/ERR: str.cat internals,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
9,22724,Mismatch in Series/DataFrame boolean ops behavior,[],open


## 使用数据库

In [89]:
import sqlite3
query = """
CREATE TABLE if not exists test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [90]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [91]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [92]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [97]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Atlanta,Georgia,1.25,6
4,Tallahassee,Florida,2.60,3
5,Sacramento,California,1.70,5
6,Atlanta,Georgia,1.25,6
7,Tallahassee,Florida,2.60,3
8,Sacramento,California,1.70,5


In [98]:
import pandas.io.sql as sql
sql.read_frame('select * from test', con)

AttributeError: module 'pandas.io.sql' has no attribute 'read_frame'

In [99]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

ModuleNotFoundError: No module named 'sqlalchemy'

## 存取MongoDB中的数据

In [100]:
#conda install pymongo
import pymongo
from pymongo import MongoClient
con = MongoClient()
con
#con = pymongo.Connection('localhost', port=27017)
#新的pymongo中取消了Connection这个方法

ModuleNotFoundError: No module named 'pymongo'

In [124]:
tweets = con.db.tweets

In [125]:
import requests, json
url = 'http://search.twitter.com/search.json?q=python%20pandas' 
data = json.loads(requests.get(url).text)
for tweet in data['results']: 
    tweets.save(tweet)

ConnectionError: HTTPConnectionPool(host='search.twitter.com', port=80): Max retries exceeded with url: /search.json?q=python%20pandas (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1268e3f50>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [ ]:
cursor = tweets.find({'from_user': 'wesmckinn'})

In [ ]:
tweet_fields = ['created_at', 'from_user', 'id', 'text'] 
result = DataFrame(list(cursor), columns=tweet_fields)